In [4]:
# Import CWI modules and point machine path to temporary path that the CWI module uses to do work
from complex_labeller import Complexity_labeller
model_path = './cwi_seq.model'
temp_path = './temp_file.txt'
model = Complexity_labeller(model_path, temp_path)

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` 

In [3]:
# import spaCy NLP modules and libraries
import spacy
spacy.require_gpu()
import scispacy
from scispacy.linking import EntityLinker
nlp_med = spacy.load("en_core_sci_scibert")
nlp_med.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib
/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/spacy/util.py:732: UserWarning: [W095] Model 'en_core_sci_scibert' (0.4.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/karl/anaconda3/envs/rapids-21.08/lib/pyth

In [4]:
# Import custom readability scoring modules for spacy
import readability_v_ks
nlp_read = readability_v_ks.spacy.load("en_core_web_sm")
nlp_read.add_pipe('readability')

In [5]:
# Pandas deserves its own import cell - multiple packages rely on it
import pandas as pd

In [6]:
# Import TFIDF modules
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
# Import GingerIt modules
from gingerit.gingerit import GingerIt # pip install gingerit
#import pandas as pd
import pysbd, re # pip install pysbd
segmentor = pysbd.Segmenter(language="en", clean=False)
subsegment_re = r'[^;:\n•]+[;,:\n•]?\s*'


In [8]:
# The main function of this script which calls many functions below to "automatically translate" medical text
def automatic_translation(document, new_document, find_complex_words, sub_umls_hypernym, semantic_df):
    cw_list = find_complex_words(document)
    new_document = sub_umls_hypernym(document, new_document, semantic_df, cw_list)
    return new_document, cw_list

#tui_word
#token
#token_string_list
#test_document = document
#new_document = new_document.lower()
#new_document = runGinger(new_document)

In [9]:
# Function the finds complex word in a sentence with the CWI module from the 2019 paper
def find_complex_words(test_document):
    Complexity_labeller.convert_format_string(model, test_document)
    dataframe = Complexity_labeller.get_dataframe(model)

    # Access binary labeling information from the dataframe format:
    cw_list = list(zip(dataframe['sentences'].values[0], dataframe['labels'].values[0], dataframe['probs'].values[0]))

    # get_bin_labels returns the binary complexity labels for the input
    # bin_label_list = Complexity_labeller.get_bin_labels(model)

    # The `get_prob_labels` method returns the probability of each token belonging to the complex class.
    # prob_label_list = Complexity_labeller.get_prob_labels(model)
    return cw_list

In [10]:
# Function to substitute out complex words with hypernyms from UMLS semantic tree - we need to fix this to use hypernymy look up table once complete
# This requires the sentence to have first been passed through the "find_complex_words" function as it needs the complex words list aka cw_list
def sub_umls_hypernym(document, new_document, semantic_df, cw_list):
    doc_med = nlp_med(document)
    linker = nlp_med.get_pipe("scispacy_linker")
    for token in doc_med.ents:
        token_string = str(token)
        token_string_list = token_string.split(' ')
        tui = ''

        try:
            umls_data = linker.kb.cui_to_entity[token._.kb_ents[0][0]]
            umls_data = str(umls_data).split('\n')
            tui = umls_data[2]
        except:
            pass

        try:
            tui_split = tui.split(' ')
            for tui in tui_split:
                try:
                    tui = tui.strip(',')
                    tui_word_row = semantic_df.loc[semantic_df['TUI_ID'] == tui] # pandas sem df find row with matching tui
                    tui_word = tui_word_row.iloc[0,0] # tui_word is a single row 3 column df, select row 0 (only row), column 0 (word)
                    #tui_tree_level = tui_word_row.iloc[0,2] # same as above but the tree level is in third cell, aka index 2, of each row
                    tui_word = str(tui_word).lstrip()
                except:
                    pass

        except IndexError or UnboundLocalError:
            pass

        for i in cw_list:
            cw = i[0]
            cw_bin_complexity = i[1]

            if cw in token_string_list and cw_bin_complexity == 1:
                try:
                    if tui_word != '':
                        new_document = new_document.replace(str(token), tui_word.lower())
                except:
                    pass
    return new_document

In [11]:
# Call the functions from the customized readability script Saadh and I wrote
def grade_the_documents(document, new_document):
    doc_med = nlp_read(document)
    og_fk_score = str(doc_med._.flesch_kincaid_reading_ease)
    og_fk_grade = str(doc_med._.flesch_kincaid_grade_level)
    og_dc_score = str(doc_med._.dale_chall)
    og_smog_score = str(doc_med._.smog)
    og_cl_score = str(doc_med._.coleman_liau_index)
    og_ar_index = str(doc_med._.automated_readability_index)
    og_forcast = str(doc_med._.forcast)

    #new_document = new_document['result'].replace('_',' ')
    doc_new = nlp_read(new_document)
    new_fk_score = str(doc_new._.flesch_kincaid_reading_ease)
    new_fk_grade = str(doc_new._.flesch_kincaid_grade_level)
    new_dc_score = str(doc_new._.dale_chall)
    #new_smog_score = str(doc_new._.smog)
    new_cl_score = str(doc_new._.coleman_liau_index)
    new_ar_index = str(doc_new._.automated_readability_index)
    #new_forcast = str(doc_new._.forcast)
    return og_fk_score, new_fk_score, og_fk_grade, new_fk_grade, og_dc_score, new_dc_score, og_cl_score, new_cl_score, og_ar_index, new_ar_index

In [12]:
# Grammar checker - we use GingerIt - this function requires splitting the sentence up as it is the free API version
def runGinger(par):
    fixed = []
    for sentence in segmentor.segment(par):
        if len(sentence) < 300:
            fixed.append(GingerIt().parse(sentence)['result'])
        else:
            subsegments = re.findall(subsegment_re, sentence)
            if len(subsegments) == 1 or any(len(v) < 300 for v in subsegments):
                fixed.append(sentence)
            else:
                res = []
                for s in subsegments:
                    res.append(GingerIt().parse(s)['result'])
                fixed.append("".join(res))
    return " ".join(fixed)


In [13]:
import difflib as dl
def sentencediff(sent1, sent2):
    word_list = []
    s1 = sent1.split(' ')
    s2 = sent2.split(' ')
    dl.context_diff(s1, s2)
    for diff in dl.context_diff(s1, s2):
        if diff.startswith('!'):
            word = str(diff.strip('! '))
            word_list.append(word)
    final_word_list = " ".join(word_list)
    return final_word_list

In [15]:
# Create dataframe for umls semantic tree for least specific hypernyms - most complexity reduction
columns = ['root_hypernym','TUI_ID','tree_level']
semantic_df = pd.read_csv(
    "./datasets/modified_umls_semantic_type_tree.tsv",
    sep="\t",
    names=columns
)
semantic_wordphrases = semantic_df['root_hypernym'].values.tolist()
semantic_wordphrases = [x.lower() for x in semantic_wordphrases]
print(semantic_df)
print(semantic_wordphrases)

                root_hypernym TUI_ID  tree_level
0                       event   T051         1.0
1                    activity   T052         2.0
2                    behavior   T053         3.0
3                    behavior   T054         4.0
4                    behavior   T055         4.0
..                        ...    ...         ...
122           group-attribute   T102         3.0
123      intellectual-product   T170         3.0
124                regulation   T089         4.0
125            classification   T185         4.0
126                  language   T171         3.0

[127 rows x 3 columns]
['event', '  activity', '    behavior', '      behavior', '      behavior', '    activity', '    work-activity', '      healthcare-activity', '        lab-procedure', '        test-procedure', '        medical-procedure', '      research', '        molecular-research', '      regulatory-act', '      educational-act', '    machine-process', '  process', '    injury', '    human-effect',

In [42]:
#test_document = "an organothiophosphate insecticide."
f = open("./datasets/definitions_output.txt", "a+")
for line in open("./datasets/UMLS/MRDEF.RRF", 'r'):
    #print(line)
    document = str(line.strip('\n'))
    source = str(line.split('|')[4])
    if source == 'MSH':
        document = str(line.split('|')[5])
        test_document = document
        count = 0
        new_document = test_document.lower()
        new_document, cw_list = automatic_translation(test_document, new_document, find_complex_words, sub_umls_hypernym, semantic_df)
        try:
            new_document = runGinger(new_document)
        except KeyError:
            new_document = "ERROR WITH GINGERIT - INVALID RESULT|"+new_document
        cw_new_list = find_complex_words((new_document))
        og_fk_score, new_fk_score, og_fk_grade, new_fk_grade, og_dc_score, new_dc_score, og_cl_score, new_cl_score, og_ar_index, new_ar_index = grade_the_documents(test_document, new_document)
        word_changes = sentencediff(test_document.lower(), new_document)
        #orig text|sub text|words changed|cw orig|cw sub|orig fk|sub fk|orig grad|sub grade|orig dc|sub dc|orig cl|sub cl|orig ari|sub ari|
        final_row = "RESULT: {}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(test_document.strip('\n'), new_document.strip('\n'), word_changes.strip('\n'), og_fk_score,new_fk_score,og_fk_grade,new_fk_grade, og_dc_score,new_dc_score,og_cl_score,new_cl_score,og_ar_index,new_ar_index)
        #f.write(final_row+'\n')
        f.write(final_row+'\n')
        count += 1


    else:
        pass


f.close()

/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [ ]:
#test_document = "an organothiophosphate insecticide."
f = open("./datasets/definitions_output.txt", "a+")

        new_document, cw_list = automatic_translation(test_document, new_document, find_complex_words, sub_umls_hypernym, semantic_df)
        try:
            new_document = runGinger(new_document)
        except KeyError:
            new_document = "ERROR WITH GINGERIT - INVALID RESULT|"+new_document
        cw_new_list = find_complex_words((new_document))
        og_fk_score, new_fk_score, og_fk_grade, new_fk_grade, og_dc_score, new_dc_score, og_cl_score, new_cl_score, og_ar_index, new_ar_index = grade_the_documents(test_document, new_document)
        word_changes = sentencediff(test_document.lower(), new_document)
        #orig text|sub text|words changed|cw orig|cw sub|orig fk|sub fk|orig grad|sub grade|orig dc|sub dc|orig cl|sub cl|orig ari|sub ari|
        final_row = "RESULT: {}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(test_document.strip('\n'), new_document.strip('\n'), word_changes.strip('\n'), og_fk_score,new_fk_score,og_fk_grade,new_fk_grade, og_dc_score,new_dc_score,og_cl_score,new_cl_score,og_ar_index,new_ar_index)
        #f.write(final_row+'\n')
        f.write(final_row+'\n')
        count += 1


    else:
        pass


f.close()

documents = []
for line in open("./datasets/UMLS/MRDEF.RRF", 'r'):
    #print(line)
    document = str(line.strip('\n'))
    source = str(line.split('|')[4])
    if source == 'MSH':
        document = str(line.split('|')[5])
        test_document = document
        documents.append(test_document)

delayed_results = [delayed(automatic_translation)(document,new_document=document.lower(),find_complex_words, sub_umls_hypernym, semantic_df) for document in documents]
final_results = compute(*delayed_results, scheduler="threads")





In [ ]:
def tfidf_search(query, list_to_search):
    # Get tf-idf matrix using fit_transform function
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(list_to_search) # Store tf-idf representations of all docs
    query_vec = vectorizer.transform([query])  # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
    results = cosine_similarity(X, query_vec).reshape((-1,))  # Op -- (n_docs,1) -- Cosine Sim with each doc
    print("#####################################",query,"#####################################")
    print(X.shape) # (Number of wordphrases, Number of unique words)
    print("###################################################################################")
    tfidif_results = results.argsort()[-1:][::-1]
    return tfidif_results

    #for i in results.argsort()[-1:][::-1]:
        #print(df.iloc[i, 0], "--", df.iloc[i, 1],"--", df.iloc[i, 3])







## Med7 to resolve medicines
Need to figure out how to use two versions of spacy
Can a python script switch environments?

## Create hypernymy NER to make a training set, this will allow us to train a machine learning model that automatically subs hypernyms

## Steal the nature paper methods to resolve abbreviations or figure out how to resolve them yourself

# For v2:
## Create to the point! - a module that takes an EHR note finds bottom line [diagnoses] and [plan] for such, tells patient they have [condition], which is a [definition], that they must do [plan] for.
Do this with NER based rules. If entity == label_ then return that variable.

Include a section called "Why your doctor thinks you have this." Then create NER based rules that recognize subjective, physical exam findings etc.

Thought of idea of all permutations and combinations, but again scorers really just grade based on number words, word length and sentence length.

